# Ensemble

- 앙상블이란 ?
  - 여러개의 모델을 결합하여 훨씬 강력한 모델을 생성하는 기법
  - 앙상블의 종류에는 보팅, 배깅, 부스팅, 스태킹이 있다.

- 보팅
  - 데이터를 서로 다른 알고리즘을 통해 평가하고 모든 값의 평균을 내는 것이다. 서로 다른 유형의 알고리즘으로 여러개의 모델을 만드는 것이 보팅의 특징이다.
  - 보팅은 하드보팅과 소프트 보팅으로 나뉘는데, 하드 보팅은 모델을 통해 값을 예측하고 다수결의 원칙에 의해 값(최빈값)을 선정한다. 소프트 보팅은 0이 될 확률과 1이 될 확률을 각 각 예측하고, 비율의 평균을 따져 확률이 높은것을 선택한다. 
  
- 배깅
  - 서로 다른 샘플 데이터(중복을 허용하며 랜덤 추출)를 같은 알고리즘을 사용하여 여러 개의 모델을 만들어 결과를 집계하여 값을 얻는다. 중복된 샘플을 뽑는 방법을 Bootstrap (복원 랜덤 샘플링 방식이라고도 함)이라고 한다.
  - 범주형 데이터는 투표방식으로 결과를 집계하고 연속형 데이터는 평균으로 결과를 집계한다. 보팅과 배깅을 비교했을 때, 가장 큰 차이는 같은 알고리즘을 쓰냐? 다른 알고리즘을 쓰냐?의 차이다.
  
- 부스팅
  - 여러개의 알고리즘이 순차적으로 학습-예측하면서 이전에 학습한 알고리즘의 예측이 틀린 데이터를 올바르게 예측할 수 있도록, 다음 알고리즘에 가중치를 부여하고 학습과 예측하는 방식 
  - 예측 성능이 뛰어나 앙상블 학습을 주도하며, 대표적인 부스팅 방법으로 XGBoost, LightGBM 이 있다. 
  
- 스태킹
  - 여러 알고리즘을 사용해 결과를 예측하고 그 결과를 새로운 최종 모델에게 예측하게 하는 것 

In [3]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings(action='ignore')

In [4]:
# 데이터 불러오기
path = 'https://raw.githubusercontent.com/jangrae/csv/master/iris.csv'
data = pd.read_csv(path)

In [5]:
# target확인
target ='Species'

x=data.drop(target, axis=1)
y=data.loc[:,target]

In [7]:
#정규화
x=  (x-x.min()) / (x.max() - x.min())

In [8]:
# 모듈 불러오기
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test=train_test_split(x, y, test_size=0.3, random_state=2022)

## 모델링

### 1.KNN

In [9]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report

model = KNeighborsClassifier(n_neighbors=5)

model.fit(x_train, y_train)

y_pred = model.predict(x_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[14  0  0]
 [ 0 15  0]
 [ 0  1 15]]
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        14
  versicolor       0.94      1.00      0.97        15
   virginica       1.00      0.94      0.97        16

    accuracy                           0.98        45
   macro avg       0.98      0.98      0.98        45
weighted avg       0.98      0.98      0.98        45



### 2.Decision Tree

In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report

model = DecisionTreeClassifier(max_depth=10, random_state=2022)

model.fit(x_train, y_train)

y_pred = model.predict(x_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[14  0  0]
 [ 0 13  2]
 [ 0  1 15]]
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        14
  versicolor       0.93      0.87      0.90        15
   virginica       0.88      0.94      0.91        16

    accuracy                           0.93        45
   macro avg       0.94      0.93      0.94        45
weighted avg       0.93      0.93      0.93        45



### 3.Random Forest

- Random Forest
  - 여러 tree가 모여있다는 의미로 랜덤 포레스트 라는 이름이 붙여졌다. 의사결정 트리는 학습 데이터에 과대적합되는 경향이 있는데 랜덤 포레스트가 이 문제를 해결할 수 있다. 샘플 데이터 100개를 만들면 중복이 생기게 되는데, 과적합을 막기위해 일부러 중복을 만든다. 같은 알고리즘을 사용하여 결과를 예측하고 값을 낸다. 

- 함수 : from sklearn.ensemble import RandomForestClassifier / RandomRegressor 

- Hyperparameter : n_estimators(트리의 개수) 

- Row: 랜덤하게 샘플링 

- Feature : 각 tree에서 가지를 나눌 때 기준이 되는 Feature를 랜덤하게 선정 

- 장점: 의사결정 트리의 쉽고 직관적인 장점, 앙상블 알고리즘 수행 속도가 비교적 빠름, 다양한 분야 좋은 성능 

- 단점: 하이퍼 파라미터가 많아 튜닝을 위해 많은 시간이 소요된다. 

In [13]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(max_depth=10, random_state=2022)

model.fit(x_train, y_train)

y_pred = model.predict(x_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[14  0  0]
 [ 0 15  0]
 [ 0  2 14]]
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        14
  versicolor       0.88      1.00      0.94        15
   virginica       1.00      0.88      0.93        16

    accuracy                           0.96        45
   macro avg       0.96      0.96      0.96        45
weighted avg       0.96      0.96      0.96        45



### 4. XGBoost

- XGBoost
  - 부스팅 기법을 이용하여 구현한 알고리즘 
- 함수 : from xgboost import XGBRegressor / XGBClassifier 
- Hyperparameter : max_depth(트리 depth 제한), n_estimators(반복 횟수) 

In [14]:
from xgboost import XGBClassifier

model = XGBClassifier(max_depth=10, random_state=2022, n_estimators=20)

model.fit(x_train, y_train)

y_pred = model.predict(x_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[23:20:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[[14  0  0]
 [ 0 15  0]
 [ 0  2 14]]
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        14
  versicolor       0.88      1.00      0.94        15
   virginica       1.00      0.88      0.93        16

    accuracy                           0.96        45
   macro avg       0.96      0.96      0.96        45
weighted avg       0.96      0.96      0.96        45

